In [9]:
import findspark
findspark.init('/usr/local/spark')

In [10]:
from pyspark import SparkContext, SparkConf, SQLContext
import re
from operator import add
import numpy as np

In [11]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [12]:
questionDataFrame = sqlContext.read.format("csv").options(header="true").load("test_q.csv")
answerDataFrame = sqlContext.read.format("csv").options(header="true").load("test_a.csv")
tagDataFrame = sqlContext.read.format("csv").options(header="true").load("Tags.csv")

print("finish reading source files")

finish reading source files


In [13]:
stopWordsList = sc.textFile("stopwords.txt").collect()
print("finish reading stopwords: " + str(len(stopWordsList)))

finish reading stopwords: 671


In [16]:
questionLower = questionDataFrame.rdd.map(lambda x: (x["Id"], re.sub("<.*?>", " ", (x["Title"] + x["Body"]).lower())))
questionLower = questionLower.map(lambda x: (x[0], re.sub("[^a-zA-Z0-9\']+", " ", x[1])))
print(questionLower.count())

9999


In [17]:
def f2(x): return x.split(" ")
questionPair = questionLower.flatMapValues(f2).filter(lambda x: x[1].strip() and x[1] not in stopWordsList)
print(questionPair.count())

629760


In [18]:
questionPairStage1 = questionPair.map(lambda x:(x[0] + "@" + x[1], 1)).reduceByKey(add)
print(questionPairStage1.count())

381707


In [19]:
N = questionLower.count()
print(N)

9999


In [20]:
questionPairStage2MapPhase = questionPairStage1.map(lambda x: (x[0].split("@")[1], x[0].split("@")[0] + "=" + str(x[1])))
print(questionPairStage2MapPhase.count())

381707


In [21]:
stage2Map = questionPairStage2MapPhase.countByKey()

In [22]:
questionPairStage2 = questionPairStage2MapPhase.map(lambda x: (x[0]+"@"+x[1].split("=")[0], (1 + np.log(int(x[1].split("=")[1]))) * np.log(N/stage2Map.get(x[0]))))
print(questionPairStage2.take(10))

[('written@80', 3.543813678863417), ('generation@80', 5.40357787720553), ('script@80', 3.100992784211484), ('sql@80', 2.5356789751614235), ('air@80', 9.674190590121055), ('application@80', 1.8765639713181657), ('primary@80', 8.913636887908616), ('key@80', 6.698029935286301), ('40@80', 5.258996648394422), ('fileid@80', 8.517093186415904)]


In [23]:
questionPairStage3MapPhase = questionPairStage2.map(lambda x: (x[0].split("@")[1], x[0].split("@")[0] + "=" + str(x[1])))
print(questionPairStage3MapPhase.take(10))

[('80', 'written=3.543813678863417'), ('80', 'generation=5.40357787720553'), ('80', 'script=3.100992784211484'), ('80', 'sql=2.5356789751614235'), ('80', 'air=9.674190590121055'), ('80', 'application=1.8765639713181657'), ('80', 'primary=8.913636887908616'), ('80', 'key=6.698029935286301'), ('80', '40=5.258996648394422'), ('80', 'fileid=8.517093186415904')]


In [24]:
def f3_1(a,b): return float(a) + float(b.split("=")[1]) * float(b.split("=")[1])
def f3_2(a,b): return float(a) + float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0, f3_1, f3_2)
print(questionPairStage3AggByKey.take(10))

[('90', 544.0883305047198), ('260', 2543.4435916221837), ('1070', 471.03414613546374), ('1390', 910.4975510530411), ('1600', 227.24803267591417), ('1610', 1081.6517409695803), ('1970', 322.61007756575844), ('2300', 182.61121501296992), ('2550', 786.9236221911508), ('2900', 2314.9657180436343)]


In [27]:
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()

In [28]:
questionPairStage3 = questionPairStage3MapPhase.map(lambda x: (x[0], x[1].split("=")[0] + "=" + str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))
print(questionPairStage3.take(10))

[('80', 'written=0.04474262586030945'), ('80', 'generation=0.06822318698888072'), ('80', 'script=0.03915176488172284'), ('80', 'sql=0.032014362483043256'), ('80', 'air=0.12214205635493153'), ('80', 'application=0.023692667640064488'), ('80', 'primary=0.11253964132173533'), ('80', 'key=0.0845663667881615'), ('80', '40=0.06639776826957776'), ('80', 'fileid=0.10753305573881579')]


In [29]:
num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0", "b=0", "c=0", "d=0", "e=0", "f=0", "g=0"], f4_1, f4_2)
print(questionPairStage4.take(5))

[('90', ['branching=0.46729106653741126', 'tortoisesvn=0.46729106653741126', 'merging=0.4441754038906327', 'subversion=0.2014679364163259', 'client=0.13509206697173348', 'specific=0.13419991803055165', 'g=0']), ('260', ['icard=0.3832587871541616', 'cards=0.27344168455114365', 'mygame=0.18262486559506322', 'cardlibrary=0.18262486559506322', 'currentgamestate=0.18262486559506322', 'trading=0.18262486559506322', 'onenterplay=0.18262486559506322']), ('1070', ['vsz=0.42437009205824194', 'ux=0.42437009205824194', 'hp=0.33471053528352396', 'ps=0.24505097850880594', 'e=0', 'f=0', 'g=0']), ('1390', ['pare=0.30523306204048484', 'barebones=0.28226173578437', 'definitively=0.26882437133271153', 'addressed=0.22576532966804808', 'sql=0.2005297875225649', 'fully=0.16954395100958516', 'services=0.12574014500314476']), ('1600', ['prone=0.4270487021124779', 'database=0.3443136765867007', 'copy=0.24338561268984787', 'error=0.1609920867314489', 'e=0', 'f=0', 'g=0'])]


In [30]:
outMap=questionPairStage4.collectAsMap()

In [252]:
# mock expected keywords
keywords=list(set(questionPairStage4.map(lambda x: list(map((lambda x: x.split("=")[0]), x[1]))).reduce(add)))
print(keywords)

['highlighted', 'sincerely', 'arenotequal', "test4'", 'housematriarch', 'gawk', 'taginclude', 'enclose', 'usercontrols', 'varptr', "'food'", 'workable', 'ii', 'mytestcase', 'extrabatteries', 'targetexception', 'tune', 'endinvoke', 'esteemed', 'instancecontextmode', 'managementtools', 'serialization', 'fond', 'laymen', 'shellview', 'absmaxvoltage', 'point2', 'minimizes', 'pbalance', 'aiscontentpreferred', 'htmlcontent', 'solve', 'efforts', 'input', '80', 'trickycounter', 'multp', 'strive', 'casmailbox', "51'", 'computing', 'sqlserverce', 'webqueryopen', 'newdocument', 'modellocator', 'ipad', 'corporates', 'cracking', 'lookupusername', 'ironically', 'tps', 'classfactory', 'dependancy', 'explained', 'directinput', 'hoops', 'cref', 'subgroup', '305029014037881475265618958103073425958633163441030267478942720', 'remort', "'target", 'shutterfly', 'logoff', 'viewdata', 'amazed', 'n1', 'dosomethingwith', 'main', 'coarse', 'imagepath', '351', 'otp', 'exercises', 'avm', 'pints', 'ffmpeg', 'pbytes

In [524]:
newQuestion = sc.wholeTextFiles("new_q.txt")
newQuestion = newQuestion.map(lambda x: re.sub('\n', ' ', x[1]))
newQuestion = newQuestion.map(lambda x: re.sub('<.*?>', ' ', x.lower()))
newQuestion = newQuestion.map(lambda x: re.sub('[^a-zA-Z0-9\']+', ' ', x))

print(newQuestion.take(1))

[" getting back into a bit more net after a few years of not using it full time and wondering what the good unit testing packages are these days i'm familiar with nunit a few years ago and have played briefly around with ironruby with the goal of getting something like rspec going but don't know much beyond that "]


In [525]:
newQuestionKeywordsStageOne = newQuestion.flatMap(lambda x: x.lower().split(" ")).filter(lambda x: x != '' and x not in stopWordsList).map(lambda x: (x, 1)).reduceByKey(add).map(lambda x: (x[0], 'new=' + str(x[1])))
print(newQuestionKeywordsStageOne.take(10))

[('bit', 'new=1'), ('net', 'new=1'), ('years', 'new=2'), ('full', 'new=1'), ('time', 'new=1'), ('wondering', 'new=1'), ('good', 'new=1'), ('unit', 'new=1'), ('testing', 'new=1'), ('packages', 'new=1')]


In [526]:
newQuestionKeywordsStageTwoMap = newQuestionKeywordsStageOne.union(questionPairStage2MapPhase).countByKey()

newQuestionKeywordsStageTwo = newQuestionKeywordsStageOne.map(lambda x: (x[0], (1 + np.log(int(x[1].split("=")[1]))) * np.log((N + 1)/newQuestionKeywordsStageTwoMap.get(x[0]))))
print(newQuestionKeywordsStageTwo.take(10))

[('bit', 3.1820618517454844), ('net', 1.8263509139976741), ('years', 7.516990611554189), ('full', 3.7679226614543895), ('time', 2.1404662435176105), ('wondering', 3.2163789446696134), ('good', 2.4721878773802257), ('unit', 4.074541934925921), ('testing', 3.713172146682981), ('packages', 5.2030071867437115)]


In [527]:
newQuestionKeywordsStageThreeSquare = newQuestionKeywordsStageTwo.map(lambda x: np.square(x[1])).reduce(add)

newQuestionKeywordsStageThree = newQuestionKeywordsStageTwo.map(lambda x: (x[0], x[1]/np.sqrt(newQuestionKeywordsStageThreeSquare)))
print(newQuestionKeywordsStageThree.take(50))

[('bit', 0.14971800189001935), ('net', 0.08593095368141608), ('years', 0.3536791134247081), ('full', 0.1772831196978871), ('time', 0.10071027655125685), ('wondering', 0.15133264259238544), ('good', 0.11631798706083112), ('unit', 0.191709748438603), ('testing', 0.1747070737076029), ('packages', 0.24480474488306264), ('days', 0.21051117802866523), ("i'm", 0.056257210185877086), ('familiar', 0.23551885997163446), ('nunit', 0.27828109205353274), ('ago', 0.2302117613693949), ('played', 0.2838228576057459), ('ironruby', 0.38166188373800974), ('goal', 0.2295885651601491), ('rspec', 0.3250142096851862), ('going', 0.14815693746377379)]


In [528]:
newQuestionKeywordsStageFour = newQuestionKeywordsStageThree.filter(lambda x: x[0] in keywords).sortBy(lambda x: -x[1])
print(newQuestionKeywordsStageFour.take(50))

[('ironruby', 0.38166188373800974), ('years', 0.3536791134247081), ('rspec', 0.3250142096851862), ('played', 0.2838228576057459), ('nunit', 0.27828109205353274), ('packages', 0.24480474488306264), ('familiar', 0.23551885997163446), ('ago', 0.2302117613693949), ('goal', 0.2295885651601491), ('days', 0.21051117802866523), ('unit', 0.191709748438603), ('full', 0.1772831196978871), ('testing', 0.1747070737076029), ('wondering', 0.15133264259238544), ('bit', 0.14971800189001935), ('going', 0.14815693746377379), ('good', 0.11631798706083112), ('time', 0.10071027655125685), ('net', 0.08593095368141608), ("i'm", 0.056257210185877086)]


In [529]:
# get from buckets

In [530]:
# mock expected questions inside buckets <Qi, sum(Ai)>
candidateSimilarQuestions = questionDataFrame
print(candidateSimilarQuestions.count())

9999


In [531]:
candidateSimilarQuestions = candidateSimilarQuestions.rdd.map(lambda x: (x["Id"], re.sub("<.*?>", " ", (x["Title"] + x["Body"]).lower())))
candidateSimilarQuestions = candidateSimilarQuestions.map(lambda x: (x[0], re.sub("[^a-zA-Z0-9\']+", " ", x[1])))
candidateSimilarQuestions = newQuestion.map(lambda x: ['0', x]).union(candidateSimilarQuestions)
candidateSimilarQuestions = candidateSimilarQuestions.map(lambda x: (x[0], list(filter(None, x[1].split(" ")))))
print(candidateSimilarQuestions.count())

10000


In [532]:
print(candidateSimilarQuestions.take(2))

[('0', ['getting', 'back', 'into', 'a', 'bit', 'more', 'net', 'after', 'a', 'few', 'years', 'of', 'not', 'using', 'it', 'full', 'time', 'and', 'wondering', 'what', 'the', 'good', 'unit', 'testing', 'packages', 'are', 'these', 'days', "i'm", 'familiar', 'with', 'nunit', 'a', 'few', 'years', 'ago', 'and', 'have', 'played', 'briefly', 'around', 'with', 'ironruby', 'with', 'the', 'goal', 'of', 'getting', 'something', 'like', 'rspec', 'going', 'but', "don't", 'know', 'much', 'beyond', 'that']), ('80', ['sqlstatement', 'execute', 'multiple', 'queries', 'in', 'one', 'statement', "i've", 'written', 'a', 'database', 'generation', 'script', 'in', 'sql', 'and', 'want', 'to', 'execute', 'it', 'in', 'my', 'adobe', 'air', 'application', 'create', 'table', 'trole', 'roleid', 'integer', 'primary', 'key', 'rolename', 'varchar', '40', 'create', 'table', 'tfile', 'fileid', 'integer', 'primary', 'key', 'filename', 'varchar', '50', 'filedescription', 'varchar', '500', 'thumbnailid', 'integer', 'fileformati

In [533]:
# generate shingles
shingleLength = 5
candidateQuestionShingles = candidateSimilarQuestions.map(lambda x: (x[0], [" ".join(x[1][i: i + shingleLength]) for i in range(len(x[1]) - shingleLength + 1)]))
print(candidateQuestionShingles.take(2))

[('0', ['getting back into a bit', 'back into a bit more', 'into a bit more net', 'a bit more net after', 'bit more net after a', 'more net after a few', 'net after a few years', 'after a few years of', 'a few years of not', 'few years of not using', 'years of not using it', 'of not using it full', 'not using it full time', 'using it full time and', 'it full time and wondering', 'full time and wondering what', 'time and wondering what the', 'and wondering what the good', 'wondering what the good unit', 'what the good unit testing', 'the good unit testing packages', 'good unit testing packages are', 'unit testing packages are these', 'testing packages are these days', "packages are these days i'm", "are these days i'm familiar", "these days i'm familiar with", "days i'm familiar with nunit", "i'm familiar with nunit a", 'familiar with nunit a few', 'with nunit a few years', 'nunit a few years ago', 'a few years ago and', 'few years ago and have', 'years ago and have played', 'ago and ha

In [534]:
from hashlib import sha1
from datasketch.minhash import MinHash
import hashlib

In [535]:
candidateQuestionHashedShingles = candidateQuestionShingles.map(lambda x: (x[0], [hashlib.sha256(x[1][i].encode()).hexdigest() for i in range(len(x[1]) - 1)]))
print(candidateQuestionHashedShingles.take(2))


[('0', ['b43b3c728c9b7cd229b104492a1dce2280b39726b7b3ea0c34a6eabafec42728', 'bd8d6bebde2732cdb8b0ac10d75c35d99257dd2e6fb5a52085f6e3a3912bd919', '1fa51df5b6213225354c47ccdeace059cd4cef534539b9757a9965f1fab26358', '30c0b596b47f5ff3621614456be49c0719a4ec808dd7122a9fc6bc0eb491e412', '354f2137ec9e5ed6f6b660628d30b55d6a4c90cf1f071b69bcfe5216eadc9816', '578b174926276ee7e224c5e3f291c1386b4e5c8f0647361ab2e6cb2822e91494', 'd13db448f8699f7f6b0d2f676f8e7273b2f2f42b932843452a51833862c55c54', '72cf4b21af9a6ffd02677b192adc7eee1a74dde8a9e4319a1215f27a025185b5', 'ee6a92e13b57cb845cd04346a776fb64fb78798f75fc9e0f0a175ca1d9e8fc7d', '05f7dc2bdde725b8bcc2fdb26f8c289367c6674378d7de88ccc02b925b4dc737', '870fd3c3cfaa431a7181f750135064a0db0fdcb2033b7b206d0aa33bf87c5221', '4bc153eed95351e66c672437f8f7a1d18e41eb7c50fcca42c564eebb4da71492', 'b25748f09ead534b4786565253cadf04bf169557d47ffa71444740b14ddcb6c4', 'ab3a42adcadb0b001a8e8ee9f787c82732bf40c6e6fcbffb1835c1b991a62e73', '4b0732174d948449be67777b73ac283a215593b

In [536]:
numPerm = 128
m = {}
def minhash(x):
    m[x[1]] = MinHash(num_perm = numPerm)
    for shingle in x[0][1]:
        m[x[1]].update(shingle.encode('utf8'))
    return (x[0][0], m[x[1]])
candidateQuestionMinhash = candidateQuestionHashedShingles.zipWithIndex().map(minhash)

In [537]:
newQuestionMinhash = candidateQuestionMinhash.take(1)[0][1]
candidateQuestionMinhash = candidateQuestionMinhash.filter(lambda x: x[0] != '0')

In [538]:
threshold = 0.01
lsh = MinHashLSH(threshold = threshold, num_perm = numPerm)

with lsh.insertion_session() as session:
    for x in candidateQuestionMinhash.collect():
        session.insert(x[0], x[1])
    #candidateQuestionMinhash.foreach(lambda x: session.insert(x[0], x[1]))
similarQuestions = lsh.query(newQuestionMinhash)
print(similarQuestions)

['88190', '1760']


In [578]:
similarQuestionMinhash = candidateQuestionMinhash.filter(lambda x: x[0] in similarQuestions)
similarQuestionJaccardSimilarity = similarQuestionMinhash.map(lambda x: (x[0], newQuestionMinhash.jaccard(x[1]))).collectAsMap()
print(similarQuestionJaccardSimilarity)

{'1760': 0.6484375, '88190': 0.0078125}


In [622]:
similarQuestionDataFrame = questionDataFrame.filter(questionDataFrame.Id.isin(similarQuestions))
similarQuestionDataFrame = similarQuestionDataFrame.join(answerDataFrame, questionDataFrame.Id == answerDataFrame.ParentId).select(questionDataFrame.Id, questionDataFrame.Body, answerDataFrame.Body, answerDataFrame.Score)
print(similarQuestionDataFrame.take(1))

[Row(Id='1760', Body="<p>Getting back into a bit more .NET after a few-years of not using it full-time; and wondering what the good unit testing packages are these days.</p>  <p>I'm familiar with NUnit (a few years ago); and have played briefly around with IronRuby; with the goal of getting something like rspec going; but don't know much beyond that.</p>  <p>I realize I could google for this and call it a day but I believe I'm likely to get a better and more informed response from asking a question here :-)</p>  <p>Suggestions?</p> ", Body='"<p>I have made a small example of testing a .net lib using ironRuby: <a href=""http://khebbie.dk/post/2008/08/Example-of-using-ironRubys-mini_rspec-library.aspx"" rel=""nofollow"">http://khebbie.dk/post/2008/08/Example-of-using-ironRubys-mini_rspec-library.aspx</a></p> "', Score='1')]


In [655]:
similarQuestionAnswers = similarQuestionDataFrame.rdd.map(lambda x: (x[0], ((x[1].replace("\"\"", "\'").replace("\"", ""), x[2].replace("\"\"", "\'").replace("\"", "")), int(x[3]))))
similarQuestionAnswerScoreSqrt = similarQuestionAnswers.map(lambda x: (x[0], np.square(x[1][1]))).reduceByKey(add).map(lambda x:(x[0], np.sqrt(x[1])))
similarQuestionAnswerScoreSqrt = similarQuestionAnswerScoreSqrt.collectAsMap()
print(similarQuestionAnswerScoreSqrt)

{'88190': 3.1622776601683795, '1760': 48.40454524112379}


In [656]:
similarQuestionAnswerScoreNorm = similarQuestionAnswers.map(lambda x: (x[0], (x[1][0], x[1][1]/similarQuestionAnswerScoreSqrt.get(x[0]))))
similarQuestionAnswerScoreNorm = similarQuestionAnswerScoreNorm.map(lambda x: (x[0], (x[1][0], x[1][1] * similarQuestionJaccardSimilarity.get(x[0]))))
print(similarQuestionAnswerScoreNorm.take(1))

[('1760', (("<p>Getting back into a bit more .NET after a few-years of not using it full-time; and wondering what the good unit testing packages are these days.</p>  <p>I'm familiar with NUnit (a few years ago); and have played briefly around with IronRuby; with the goal of getting something like rspec going; but don't know much beyond that.</p>  <p>I realize I could google for this and call it a day but I believe I'm likely to get a better and more informed response from asking a question here :-)</p>  <p>Suggestions?</p> ", "<p>I have made a small example of testing a .net lib using ironRuby: <a href='http://khebbie.dk/post/2008/08/Example-of-using-ironRubys-mini_rspec-library.aspx' rel='nofollow'>http://khebbie.dk/post/2008/08/Example-of-using-ironRubys-mini_rspec-library.aspx</a></p> "), 0.013396210970888266))]


In [657]:
answerCount = 10
similarQuestionAnswerSorted = similarQuestionAnswerScoreNorm.map(lambda x: x[1]).sortBy(lambda x: -x[1])
similarQuestionAnswerTop = similarQuestionAnswerSorted.take(answerCount)

In [659]:
import csv
with open('suggested_answers.csv', 'w') as suggestedAnswers:
        writer = csv.writer(suggestedAnswers, delimiter=',')
        writer.writerow(['Rank', 'Original Question', 'Answer'])
        for idx, questionAnswer in enumerate(similarQuestionAnswerTop):
            writer.writerow([idx + 1, questionAnswer[0][0], questionAnswer[0][1]])
            
for idx, questionAnswer in enumerate(similarQuestionAnswerTop):
    print("Rank: " + str(idx + 1))
    print("Original Question: " + questionAnswer[0][0])
    print("Answer: " + questionAnswer[0][1])
    print()

Rank: 1
Original Question: <p>Getting back into a bit more .NET after a few-years of not using it full-time; and wondering what the good unit testing packages are these days.</p>  <p>I'm familiar with NUnit (a few years ago); and have played briefly around with IronRuby; with the goal of getting something like rspec going; but don't know much beyond that.</p>  <p>I realize I could google for this and call it a day but I believe I'm likely to get a better and more informed response from asking a question here :-)</p>  <p>Suggestions?</p> 
Answer: <p>There are so many it's crazy.  Crazy good; I guess.</p>  <ul> <li>For the conservative types (me); <a href='http://www.nunit.org/'><strong>NUnit</strong></a> is still available and still more than capable.</li> <li>For the Microsoft-types; <a href='http://msdn.microsoft.com/en-us/vstudio/default.aspx'><strong>MSTest</strong></a> is adequate; but slow and clunky compared to Nunit.  It also lacks code coverage without paying the big bucks for 